In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.metrics import log_loss
import pickle
from tqdm import tqdm

In [3]:
train = pd.read_pickle('../../data/lgb_feature/train_lgb_baseline.pkl')
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle('../../data/lgb_feature/test_lgb_baseline.pkl')
print('test shape is: {}'.format(test.shape))
tags = pd.read_pickle('../../data/lgb_feature/tags_gesim_window3_size10.pkl')

train shape is: (1001650, 71)
test shape is: (40024, 70)


In [4]:
all_cols = list(train.columns)
for col in tqdm(all_cols):
    if 'tags' in col:
        train.drop(col,axis=1,inplace=True)
        test.drop(col,axis=1,inplace=True)
train = train.merge(tags,how='inner',on='instance_id')
print('train shape is: {}'.format(train.shape))
test = test.merge(tags,how='inner',on='instance_id')
print('test shape is: {}'.format(test.shape))

100%|██████████| 71/71 [00:01<00:00, 61.39it/s]


train shape is: (1001650, 76)
test shape is: (40024, 75)


# LGBM

In [5]:
from sklearn.model_selection import train_test_split
print_to_file = False 
test_run = False 

def get_time(timezone='America/New_York', time_format='%Y-%m-%d %H:%M:%S'):
    from datetime import datetime
    from dateutil import tz

    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz(timezone)

    utc = datetime.utcnow()

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    est = utc.astimezone(to_zone)

    return est.strftime(time_format)

import sys, time
class Logger(object):
    def __init__(self, logtofile=True, logfilename='log'):
        self.terminal = sys.stdout
        self.logfile = "{}_{}.log".format(logfilename, int(time.time()))
        self.logtofile = logtofile

    def write(self, message):
        #         self.terminal.write(message)
        if self.logtofile:
            self.log = open(self.logfile, "a")
            self.log.write('[' + get_time() + '] ' + message)
            self.log.close()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass


def divert_printout_to_file():
    sys.stdout = Logger(logfilename='logfile')

if print_to_file:
    divert_printout_to_file()  # note: comment this to use pdb

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, train_df, test_df, holdout, num_folds, submission_file_name, fe_img_name, stratified = False, debug= False, colsample=0.67, max_depth=8, num_leaves=31, min_child_samples=20, subsample=0.7, reg_lambda=0.3, lr=0.04, seed=1001, verbose=100, rounds=None, target='click'):
    print(train_df.shape, test_df.shape, holdout.shape)
    print('MEAN: train({}) vs holdout({}): '.format(len(train_df), len(holdout)), train_df[target].mean(), holdout[target].mean())
    # Divide in training/validation and test data
    if df is not None:
        train_df = df[df[target].notnull()]
        test_df = df[df[target].isnull()]
        print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
        del df
        gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
        
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    holdout_final_preds = np.zeros(holdout.shape[0])
    feature_importance_df = pd.DataFrame()
    feature_importance_gain_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in [target,'time','instance_id','index']]
    train_scores = []
    holdout_scores = []
    scores = []
    diff_val_holdout = []
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df[target])):
#         print('valid index : ',list(valid_idx)[:5])
        train_x, train_y = train_df[feats].iloc[train_idx], train_df[target].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df[target].iloc[valid_idx]
#         print('MEAN: train({}) vs valid({}): '.format(len(train_y), len(valid_y)), np.mean(train_y), np.mean(valid_y))

        clf = LGBMClassifier(
            nthread=12,
            n_estimators=30000,
            learning_rate=lr,
            silent=-1,
            verbose=-1, )
        if rounds is not None:
            clf.n_estimators = rounds
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose)
            oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats])[:, 1] 
        else:
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose, early_stopping_rounds= 200)
            oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats], num_iteration=clf.best_iteration_)[:, 1] 
            
        holdout_final_preds += holdout_preds / folds.n_splits
        score = log_loss(valid_y, oof_preds[valid_idx])
        train_score = clf.best_score_['training']['binary_logloss']
        holdout_score = log_loss(holdout[target], holdout_preds)
        diff = abs(score - holdout_score)
        best_rounds = rounds if rounds is not None else clf.best_iteration_
        print('Fold %2d [%5d] AUC : ho: %.6f / te: %.6f / tr: %.6f (diff: %.6f)' % (n_fold + 1, best_rounds, holdout_score, score,  train_score, diff))
        scores.append(score)
        train_scores.append(train_score)
        holdout_scores.append(holdout_score)
        diff_val_holdout.append(diff)
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        fold_importance_gain_df = pd.DataFrame()
        fold_importance_gain_df["feature"] = feats
        fold_importance_gain_df["importance"] = clf.booster_.feature_importance(importance_type='gain')
        fold_importance_gain_df["fold"] = n_fold + 1
        feature_importance_gain_df = pd.concat([feature_importance_gain_df, fold_importance_gain_df], axis=0)
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    holdout_roc = log_loss(holdout[target], holdout_final_preds)
    holdout_mean = np.mean(holdout_scores)
    full_te_mean = np.mean(scores)
    full_tr_mean = np.mean(train_scores)
#     print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    print('Full HO score %.6f' % holdout_roc)
    print('FULL HO mean {:.6f}, std {:.6f}'.format(holdout_mean, np.std(holdout_scores)))
    print('FULL TE mean {:.6f}, std {:.6f}'.format(full_te_mean, np.std(scores)))
    print('FULL TR mean {:.6f}, std {:.6f}'.format(full_tr_mean, np.std(train_scores)))
    print('FULL DIFF mean {:.6f}, std {:.6f}'.format(np.mean(diff_val_holdout), np.std(diff_val_holdout)))
    # Write submission file and plot feature importance
    if not debug:
        print('saving...')
        test_df['predicted_score'] = sub_preds
        test_df[['instance_id', 'predicted_score']].to_csv(submission_file_name, index= False)
#     if not print_to_file:
#         display_importances(feature_importance_df, fe_img_name)
    feature_importance_df = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    feature_importance_gain_df = feature_importance_gain_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    return feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds,holdout_final_preds

# Display/plot feature importance
def display_importances(feature_importance_df_, fe_img_name):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig(fe_img_name+'.png')


def convert_and_save_imp_df(fe_imp_df, dumpfilename):
    fe_imp_df_mean = fe_imp_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    pickle.dump(fe_imp_df_mean, open(dumpfilename,'wb'))
    
    

def select_holdout(train,test,shift=1,seed=41):
    samples_size = int(len(test) )
    assert shift in range(1,8)
    test_time_min = test.time.min()
    test_time_max = test.time.max()
    train_time_min = train.time.min()
    train_time_max = train.time.max()
    window_lower_bound = test_time_min - shift*60*60*24
    window_upper_bound = test_time_max - shift*60*60*24
    available_train = train[(train.time<=window_upper_bound) & (train.time>=window_lower_bound)]
    holdout = available_train.sample(n=samples_size,random_state=seed).copy()
    train_split_index = list(set(train.index) - set(holdout.index))
    train_split = train.iloc[train_split_index].copy()
    return train_split,holdout

In [6]:
def runlgb(train, test, holdout):
    colsamples = [0.07]#[0.1,0.15,0.2]#[0.03,0.04,0.05,0.06,0.07,0.08]
    seeds = [1001]#[300,4000,50000,600000,7000000,80000000,523445,31275479] # 20
    depth = [5]
    leaves = [16]
    min_child_sam = [20]#, 800]
    subsamples = [1]#0.8, 0.7, 0.6, 0.5, 0.4] # was 1
    reg_lambdas = [0.5]
    # lrs = lrs.tolist()
    lrs2 = [0.1]
    nfolds = 7
    rounds = [None] #[1000]#, 1300, 1600, 1900, 2200, 2500]
    for seed in seeds:
        for colsample in colsamples:
            for d in depth:
                for l in leaves:
                    for mcs in min_child_sam:
                        for subsample in subsamples:
                            for reg_lambda in reg_lambdas:
                                for lr in lrs2:
                                    for r in rounds:
                                        filename = 'fe_936_col{}_lr{}_n{}'.format(len(train.columns), lr, nfolds)
                                        print('#############################################')
                                        print(colsample, seed, d, l, mcs, subsample, reg_lambda, lr, 'nfolds:', nfolds)
                                        print('#############################################')
                                        numfeats = len(train.columns)
                                        with timer("Run LightGBM with kfold"):
                                            return kfold_lightgbm(None, train, test, holdout, nfolds, filename+'.csv', filename, colsample=colsample, verbose=None, max_depth=d, num_leaves=l, min_child_samples=mcs, subsample=subsample, reg_lambda=reg_lambda, lr=lr, seed=seed, stratified=True, rounds=r,debug=True)                                        

In [7]:
train_index = pickle.load(open(FILE.train_index.value,'rb'))
holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))
train_df = train.loc[train_index].copy()
holdout = train.loc[holdout_index].copy()
print('train shape: {}, holdout shape: {}'.format(train_df.shape,holdout.shape))
print('train mean: {}, holdout mean: {}'.format(train_df.click.mean(),holdout.click.mean()))
split_all, gain_all,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds,holdout_preds = runlgb(train_df, test, holdout)


train shape: (961626, 76), holdout shape: (40024, 76)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
#############################################
(961626, 76) (40024, 75) (40024, 76)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  381] AUC : ho: 0.427422 / te: 0.416178 / tr: 0.404988 (diff: 0.011245)
Fold  2 [  324] AUC : ho: 0.426996 / te: 0.415898 / tr: 0.406488 (diff: 0.011098)
Fold  3 [  335] AUC : ho: 0.427332 / te: 0.415563 / tr: 0.406436 (diff: 0.011769)
Fold  4 [  342] AUC : ho: 0.427020 / te: 0.415926 / tr: 0.406064 (diff: 0.011094)
Fold  5 [  376] AUC : ho: 0.426942 / te: 0.417014 / tr: 0.404837 (diff: 0.009928)
Fold  6 [  379] AUC : ho: 0.427579 / te: 0.416557 / tr: 0.404855 (diff: 0.011022)
Fold  7 [  328] AUC : ho: 0.427094 / te: 0.416177 / tr: 0.406420 (diff: 0.010917)
Full HO score 0.426533
FULL HO mean 0.427198, std 0.000227

In [34]:
train_save = train[['instance_id']].copy()
# train_save = pd.DataFrame()
train_save.loc[train_df.index,'instance_id'] = train_df.instance_id.values
train_save.loc[train_df.index,'predicted_score'] = oof_preds
train_save.loc[holdout_index,'instance_id'] = holdout.instance_id.values
train_save.loc[holdout_index,'predicted_score'] = holdout_preds

train_save.to_pickle('../../data/oof/train/lgb_libFM_ho_81col.pkl')

In [30]:
train_save.head()

,instance_id,predicted_score
0,86294719979897807,0.321591
1,2699289844928136052,0.412990
2,3117527168445845752,0.359213
3,3398484891050993371,0.028753
4,2035477570591176488,0.324440
